<a href="https://colab.research.google.com/github/minsuk-heo/tf2/blob/master/jupyter_notebooks/08.RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import Image

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, SimpleRNN, TimeDistributed
from tensorflow.keras.models import Model, Sequential
import numpy as np

# RNN Cell Practice

In [4]:
# input shape
inputs = Input(shape=(1,2))
# output shape, return state, use tanh as activation function
output, state = SimpleRNN(3, return_state=True, activation='tanh')(inputs)
model = Model(inputs=inputs, outputs=[output, state])

In [5]:
# test input
data = np.array([[ [1,2] ]])
# print output, state
output, state = model.predict(data)
print("output: ",output)
print("state: ",state)

output:  [[-0.993022   -0.8782703   0.13555866]]
state:  [[-0.993022   -0.8782703   0.13555866]]


In [6]:
# weights for input
model.layers[1].weights[0]

<tf.Variable 'simple_rnn/kernel:0' shape=(2, 3) dtype=float32, numpy=
array([[-0.9880387 , -0.12899536,  0.68050313],
       [-0.91964287, -0.61957824, -0.2720524 ]], dtype=float32)>

In [7]:
# weights for state
model.layers[1].weights[1]

<tf.Variable 'simple_rnn/recurrent_kernel:0' shape=(3, 3) dtype=float32, numpy=
array([[-0.47471285, -0.4501174 ,  0.7563346 ],
       [ 0.57666993,  0.49011993,  0.6536315 ],
       [ 0.66490567, -0.7464427 , -0.02690291]], dtype=float32)>

In [8]:
# bias
model.layers[1].weights[2]

<tf.Variable 'simple_rnn/bias:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>

# sequence tagging example

In [9]:
John = [1,0,0]
loves = [0,1,0]
Jane = [0,0,1]

X = np.array([
    [ John, loves, Jane ],
    [ Jane, loves, John ]
]).astype(np.float32)

S = [0] # subject
V = [1] # verb
O = [2] # object
y = np.array([[S, V, O], [S, V, O]]).astype(np.float32)

In [10]:
# input shape
inputs = Input(shape=(3, 3))
# output shape, return state, return sequence
output, state = SimpleRNN(3, return_state=True, return_sequences=True)(inputs)
model = Model(inputs=inputs, outputs=[output, state])

In [11]:
# print output, state
output, state = model.predict(X)

In [12]:
print("John loves Jane: ",output[0])
print("Jane loves John: ",output[1])

John loves Jane:  [[ 0.7114806   0.00293278 -0.4700191 ]
 [-0.07926637 -0.4950598   0.25483623]
 [-0.30115584  0.65368456  0.63261586]]
Jane loves John:  [[-0.34041515  0.23454879  0.7097573 ]
 [ 0.8615997  -0.00218594  0.6958208 ]
 [ 0.2863685   0.43382952  0.29268172]]


In [13]:
# the state value is same with the last output
print("John loves Jane: state: ",state[0])
print("Jane loves John: state: ",state[1])

John loves Jane: state:  [-0.30115584  0.65368456  0.63261586]
Jane loves John: state:  [0.2863685  0.43382952 0.29268172]


In [14]:
model = Sequential()
model.add(SimpleRNN(3, input_shape=(3, 3), return_sequences=True))
model.add(TimeDistributed(Dense(3, activation="softmax")))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
print(model.summary())
# train, takes 30sec, if you want to monitor progreses, change verbose=1
model.fit(X, y, epochs=2000, verbose=0)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 3, 3)              21        
_________________________________________________________________
time_distributed (TimeDistri (None, 3, 3)              12        
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
result = model.predict(X, verbose=0)

In [16]:
# check the result

# 0 : Subject
# 1 : Verb
# 2 : Object
np.argmax(result, axis=1)

array([[0, 1, 2],
       [0, 1, 2]])

# Sentence Classification
classify movie review into positive or negative.

In [17]:
from IPython.display import Image

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pandas as pd

tf.random.set_seed(1)
np.random.seed(1)

In [18]:
movie_reviews = [
         {'review': 'this is the best movie', 'sentiment': 'positive'},
         {'review': 'i recommend you watch this movie', 'sentiment': 'positive'},
         {'review': 'it was waste of money and time', 'sentiment': 'negative'},
         {'review': 'the worst movie ever', 'sentiment': 'negative'}
    ]
df = pd.DataFrame(movie_reviews)

In [19]:
df

,review,sentiment
0,this is the best movie,positive
1,i recommend you watch this movie,positive
2,it was waste of money and time,negative
3,the worst movie ever,negative


In [20]:
def get_vocab2int(df):
    d = {}
    vocab = set()
    df['review'].str.split().apply(vocab.update)
    for idx, word in enumerate(vocab):
        d[word] = idx
    return d

vocab2_int = get_vocab2int(df)
vocab_size = len(vocab2_int)

In [21]:
# encode words into integer
reviews = df['review'].tolist()
encoded_reviews = []
for review in reviews:
    tokens = review.split(" ")
    review_encoding = []
    for token in tokens:
        review_encoding.append(vocab2_int[token])
    encoded_reviews.append(review_encoding)

In [22]:
# encoded reviews
print(encoded_reviews[0])
print(encoded_reviews[1])
print(encoded_reviews[2])
print(encoded_reviews[3])

[13, 9, 11, 12, 15]
[0, 10, 16, 6, 13, 15]
[7, 3, 17, 5, 14, 8, 2]
[11, 4, 15, 1]


In [23]:
def get_max_length(df):
    max_length = 0
    for row in df['review']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

# max_length is used for max sequence of input
max_length = get_max_length(df)

In [24]:
# if review is short, fill in zero padding and make all sentence length to be same as max_length
padded_reviews_encoding = pad_sequences(encoded_reviews, maxlen=max_length, padding='post')

In [25]:
sentiments = df['sentiment'].tolist()
def sentiment_encode(sentiment):
    if sentiment == 'positive':
        return [1,0]
    else:
        return [0,1]

# encoded sentiment
encoded_sentiment = [sentiment_encode(sentiment) for sentiment in sentiments]

In [26]:
# RNN model
model = Sequential()
model.add(Embedding(vocab_size, 3, input_length=max_length))
model.add(SimpleRNN(32))
model.add(Dense(2, activation='softmax'))

In [27]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [28]:
train_X = np.array(padded_reviews_encoding)
train_Y = np.array(encoded_sentiment)

In [29]:
print('Train...')
model.fit(train_X, train_Y,epochs=50)

Train...
Train on 4 samples
Epoch 1/50
4/4 [==============================] - 2s 462ms/sample - loss: 0.6903 - accuracy: 0.2500
Epoch 2/50
4/4 [==============================] - 0s 3ms/sample - loss: 0.6840 - accuracy: 0.5000
Epoch 3/50
4/4 [==============================] - 0s 3ms/sample - loss: 0.6778 - accuracy: 0.7500
Epoch 4/50
4/4 [==============================] - 0s 6ms/sample - loss: 0.6714 - accuracy: 0.7500
Epoch 5/50
4/4 [==============================] - 0s 7ms/sample - loss: 0.6647 - accuracy: 0.7500
Epoch 6/50
4/4 [==============================] - 0s 6ms/sample - loss: 0.6576 - accuracy: 1.0000
Epoch 7/50
4/4 [==============================] - 0s 4ms/sample - loss: 0.6501 - accuracy: 1.0000
Epoch 8/50
4/4 [==============================] - 0s 5ms/sample - loss: 0.6421 - accuracy: 1.0000
Epoch 9/50
4/4 [==============================] - 0s 4ms/sample - loss: 0.6336 - accuracy: 1.0000
Epoch 10/50
4/4 [==============================] - 0s 3ms/sample - loss: 0.6245 - accura

In [30]:
score, acc = model.evaluate(train_X, train_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

4/1 - 0s - loss: 0.0076 - accuracy: 1.0000
Test score: 0.007553538773208857
Test accuracy: 1.0
